In [1]:
%run mobility_beam.py

/home/jupyter/venv/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery.py:1421: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  experiments = p.options.view_as(DebugOptions).experiments or []
INFO:apache_beam.runners.direct.direct_runner:Running pipeline with DirectRunner.
INFO:apache_beam.internal.gcp.auth:Setting socket default timeout to 60 seconds.
INFO:apache_beam.internal.gcp.auth:socket default timeout is 60.0 seconds.
INFO:oauth2client.transport:Attempting refresh to obtain initial access_token
INFO:apache_beam.io.gcp.bigquery_tools:Using location 'US' from table <TableReference
 datasetId: 'covid_staging'
 projectId: 'corvid-276516'
 tableId: 'googleMobility'> referenced by query SELECT * FROM covid_staging.googleMobility ORDER BY date, country_region limit 100
INFO:apache_beam.io.gcp.bigquery_tools:Created table corvid-276516.covid_modeled.mobility_beam with schema <TableSchema
 fields: [<T

In [6]:
%run mobility_df.py

/home/jupyter/venv/lib/python3.7/site-packages/apache_beam/runners/dataflow/dataflow_runner.py:740: BeamDeprecationWarning: BigQuerySink is deprecated since 2.11.0. Use WriteToBigQuery instead.
  kms_key=transform.kms_key))
INFO:apache_beam.runners.dataflow.internal.apiclient:Starting GCS upload to gs://covid-bucket19/staging/format-codes--df.1588906237.924080/pipeline.pb...
INFO:oauth2client.transport:Refreshing due to a 401 (attempt 1/2)
INFO:apache_beam.runners.dataflow.internal.apiclient:Completed GCS upload to gs://covid-bucket19/staging/format-codes--df.1588906237.924080/pipeline.pb in 0 seconds.
INFO:apache_beam.runners.portability.stager:Downloading source distribution of the SDK from PyPi
INFO:apache_beam.runners.portability.stager:Executing command: ['/home/jupyter/venv/bin/python', '-m', 'pip', 'download', '--dest', '/tmp/tmpnsf26zus', 'apache-beam==2.19.0', '--no-deps', '--no-binary', ':all:']
INFO:apache_beam.runners.portability.stager:Staging SDK sources from PyPI to gs:/

In [9]:
%%bigquery
CREATE OR REPLACE TABLE covid_modeled.geoDist AS
SELECT geo_id AS code, countries_and_territories AS country, date, daily_confirmed_cases AS daily_cases, 
daily_deaths, confirmed_cases AS cases, deaths, ROUND(daily_deaths/(pop_data_2018/100000),5) AS deaths_per_100000, 
ROUND(daily_confirmed_cases/(pop_data_2018/100000),5) AS cases_per_100000, pop_data_2018 AS pop, cc.continent
FROM covid_staging.covidGeoDist
LEFT JOIN covid_staging.countryContinent c 
ON geo_id = country
LEFT JOIN covid_staging.contCode cc
ON c.continent = code 
ORDER BY date, country

""


Now that the data flow has been run on googleMobility to transform it to covid_modeled.mobility_beam, there should be referential integrity between the countries in mobility_beam and geoDist. 
geoDist will be the transformed version of covidGeoDist, with a couple of new calculated fields (deaths per capita and cases per capita), continent, and cleaner columns. 
Check that code and date form a composite primary key in geoDist. Verify the referential integrity between the two tables.

In [18]:
%%bigquery
SELECT COUNT(*)
FROM covid_modeled.geoDist

,f0_
0,15282


In [20]:
%%bigquery
SELECT COUNT (*)
FROM 
  (SELECT DISTINCT code, date
  FROM covid_modeled.geoDist)

,f0_
0,15282


In [11]:
%%bigquery
SELECT DISTINCT m.code
FROM covid_modeled.mobility_beam m
WHERE m.code NOT IN 
  (SELECT g.code 
  FROM covid_modeled.geoDist g)

,code


Referential integrity has been verified. Check for a composite primary key in mobility_beam

In [15]:
%%bigquery
SELECT COUNT(*)
FROM covid_modeled.mobility_beam

,f0_
0,300747


In [16]:
%%bigquery
SELECT COUNT(*)
FROM 
  (SELECT DISTINCT country, date
   FROM covid_modeled.mobility_beam)

,f0_
0,9956


So there are many duplicate country, date value pairs. How can this be? It is because of the sub_region_1 and sub_region_2 adding a further level of granularity to each country, date pair. Some countries have been divided up further into sub regions. However, geoDist is at the country level. In order to raise mobility_beam to this level, use group by to take an average across the whole country for each date.

In [5]:
%%bigquery
CREATE OR REPLACE TABLE covid_modeled.mobility AS
SELECT code, country, date, ROUND(AVG(average_change)/100,2) AS average_change, ROUND(AVG(retail_and_recreation)/100,2) AS retail_and_recreation, 
ROUND(AVG(grocery_and_pharmacy)/100,2) AS grocery_and_pharmacy, ROUND(AVG(parks)/100,2) AS parks, ROUND(AVG(transit_stations)/100,2) AS transit_stations, 
ROUND(AVG(workplaces)/100,2) AS workplaces, ROUND(AVG(residential)/100,2) AS residential
FROM covid_modeled.mobility_beam
GROUP BY code, country, date
ORDER BY date, country

""


In [23]:
%%bigquery
SELECT COUNT (*)
FROM covid_modeled.mobility

,f0_
0,9956


In [24]:
%%bigquery
SELECT COUNT(*)
FROM 
  (SELECT DISTINCT code, date
  FROM covid_modeled.mobility)

,f0_
0,9956
